## fastjinja:

I wanted to use Jinja templates with FastHTML to separate front-end and back-end concerns, but this wasn't directly supported. So I created `fastjinja` to bridge that gap.

Maybe I am too used on the old way of programing but I find very confusing to have the back-end and front-end developped at the same place. In the example of lesson 4-5, we can see the fast iteration within the notebook to develop the web-app and it is amazing. But mixing the front-end and back-end logic is for me complicated as I don't see the separation of concerned. 

Also, I pondered a bit on the phrase from Jeremy "getting better a software development is creating layers of abstractions that makes your craft easier and not more complicated" (I hope I got that one right). We are front-row spetactors of seeing fastdaiy being created and the steps to make it happen. But I found it very "when you have a hammer, everything looks like a nail", and where python is the hammer and everything needs to be pythonic. convert html code to fasthml, convert tailwind/daisy to fasthtml components. 

I am familiar with Jinja template for front end, and for the moment I think I prefer that way of working. I can keep the separation of concern between front-end and back-end. I can have someone contributing on the html templates and not having to be a Python user (maybe that person should learn Python anyway). 

I have explored how I could get Jinja templates how to fit into the literrate programing, fasthml way of SolveIt, the same way daisy is being explored. I present you `fastjinja`.

*When skiming throught hypermedia.systems, I was also very glad to see they are using jinja! Maybe next step will be re-writing hypermedia system with fasthtml and jinja.*




In [ ]:
#| default_exp core


## Writing Jinja from notebook

In [ ]:
#| export
import jinja2 as jinja2
from fasthtml.common import *
from fasthtml.jupyter import *


Template from jinja2 conflict with another Template. That's why I import jinja2 as jinja2

In [ ]:
app = FastHTML()
rt = app.route

In [ ]:
jinja_template = """<div> {{ name }} </div>
"""
jinja_template

'<div> {{ name }} </div>\n'

In [ ]:
template = jinja2.Template(jinja_template)
template

<Template memory:7d7731239c70>

In [ ]:
template.render(name="Yann")

'<div> Yann </div>'

In [ ]:
@rt
def jinja_ex(name):
    return template.render(name=name)

jinja_ex(name="Yann")

'<div> Yann </div>'

In [ ]:
jinja_template2 = """<ul>
  {% for user in users %}
    <li><a href="{{ user.url }}">{{ user.username }}</a></li>
  {% endfor %}
  </ul>
"""
jinja_template2

'<ul>\n  {% for user in users %}\n    <li><a href="{{ user.url }}">{{ user.username }}</a></li>\n  {% endfor %}\n  </ul>\n'

In [ ]:
template2 = jinja2.Template(jinja_template2)
template2

<Template memory:7d7732dd6240>

In [ ]:
users_ex = [{"username":"Yann", "url":"yann@yann.yann"}, {"username":"Pierre", "url":"pierre@pierre.pierre"}]
users_ex

[{'username': 'Yann', 'url': 'yann@yann.yann'},
 {'username': 'Pierre', 'url': 'pierre@pierre.pierre'}]

In [ ]:
@rt
def jinja_ex2(users):
    return template2.render(users=users)

jinja_ex2(users=users_ex)

'<ul>\n  \n    <li><a href="yann@yann.yann">Yann</a></li>\n  \n    <li><a href="pierre@pierre.pierre">Pierre</a></li>\n  \n  </ul>'

### Converting Jinja to FastHTML

In [ ]:
def get_preview(app):
    return partial(HTMX, app=app, host=None, port=None)
preview = get_preview(app)

In [ ]:
c = Div(P('hi'), style='color:red')
print(c)

<div style="color:red"><p>hi</p></div>


In [ ]:
preview(c)

In [ ]:
type(c)

fastcore.xml.FT

In [ ]:
c.__dict__


{'tag': 'div',
 'children': (p(('hi',),{}),),
 'attrs': {'style': 'color:red'},
 'void_': False,
 'listeners_': []}

In [ ]:
jt = """<div style="color:red"><p>hi</p></div>"""


In [ ]:
template = jinja2.Template(jt)

In [ ]:
template.render()

'<div style="color:red"><p>hi</p></div>'

In [ ]:
preview(template.render())

In [ ]:
type(template.render())

str

I need to create the same fastcore.xml.FT element using the ft function and bs4

In [ ]:
#| export
from bs4 import BeautifulSoup

def parse_html_to_ft(html_str:str):
    """ Parse html to fastcore.xml"""
    soup = BeautifulSoup(html_str, 'html.parser')
    
    def convert_elem(elem):
        if isinstance(elem, str):
            return elem
        
        tag = elem.name
        children = [convert_elem(child) for child in elem.children]
        attrs = dict(elem.attrs)
        
        return ft(tag, *children, **attrs)
    
    return convert_elem(soup.find())

In [ ]:
ft_template = parse_html_to_ft(template.render())
ft_template

```html
<div style="color:red">
  <p>hi</p>
</div>

```

In [ ]:
preview(ft_template)

In [ ]:
type(ft_template)

fastcore.xml.FT

In [ ]:
ft_template.__dict__

{'tag': 'div',
 'children': (p(('hi',),{}),),
 'attrs': {'style': 'color:red'},
 'void_': False,
 'listeners_': []}

Now I can patch the jinja2.Template to have a module for fasthtml

In [ ]:
#| export
@patch
def ft_render(self:jinja2.Template, *args, **kargs):
    "parse html to fastcore.xml.ft and render"
    return parse_html_to_ft(self.render(*args, **kargs))

I am checking with my previous example also.

In [ ]:
preview(template.ft_render())

In [ ]:
jinja_template = """<div> {{ name }} </div>
"""
jinja_template

'<div> {{ name }} </div>\n'

In [ ]:
template = jinja2.Template(jinja_template)
template

<Template memory:7d7730a2ef60>

In [ ]:
preview(template.render(name="Yann"))

In [ ]:
preview(template.ft_render(name="Yann"))

## Reproducing (testing?)

### Writing SolveIt in SolveIt with Jinja

I will test some the first steps of lesson 4

#### Interactivity

In [ ]:
from fastcore.utils import *
from fasthtml.common import *
from fasthtml.jupyter import *
import fasthtml.components as fc
import httpx

In [ ]:
daisy_hdrs = (
    Link(href='https://cdn.jsdelivr.net/npm/daisyui@5', rel='stylesheet', type='text/css'),
    Script(src='https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4'),
    Link(href='https://cdn.jsdelivr.net/npm/daisyui@5/themes.css', rel='stylesheet', type='text/css')
)

In [ ]:
app = FastHTML(hdrs=daisy_hdrs)
rt = app.route

In [ ]:
srv = JupyUvi(app=app)

In [ ]:
def get_preview(app):
    return partial(HTMX, app=app, host=None, port=None)
preview = get_preview(app)

In [ ]:
@rt
def proc_btn(): return f'hi'

@rt
def testbtn():
    return Div(
        Button('click me', hx_post=proc_btn, id='btn', hx_target='#test', cls='btn'),
        Div(id='test')
    )

In [ ]:
preview(testbtn())

Tip: There is https://h2f.answer.ai/ to convert html to fasthtml. To convert fasthml to html just run the code and copy. It will not work for everything but that's a good start.

In [ ]:
Div(
        Button('click me', hx_post=proc_btn, id='btn', hx_target='#test', cls='btn'),
        Div(id='test')
    )

```html
<div>
<button hx-post="/proc_btn" hx-target="#test" id="btn" class="btn" name="btn">click me</button>  <div id="test"></div>
</div>

```

In [ ]:
testbtn_template = jinja2.Template("""<div>
<button hx-post="/proc_btn" hx-target="#test" id="btn" class="btn" name="btn">click me</button>  <div id="test"></div>
</div>
""")
testbtn_template

<Template memory:7d77303e4f80>

In [ ]:
@rt
def testbtn_jinja():
    return testbtn_template.ft_render()

In [ ]:
preview(testbtn_jinja())

#### Getting data in a route

In [ ]:
@rt
def getinp(): return Input(type='text', placeholder='Type here')

In [ ]:
preview(getinp)

In [ ]:
getinp_template = jinja2.Template("<input type='text' placeholder='Type here'>")

In [ ]:
@rt
def getinp_jinja():
    return getinp_template.ft_render()

In [ ]:
preview(getinp_jinja())

In [ ]:
def mk_comp(name, compcls):
    comp = getattr(fc, name)
    def wrapper(*args, cls='', **kwargs): return comp(*args, cls=f'{compcls} {cls}', **kwargs)
    globals()[name] = wrapper


mk_comp('Button', 'btn')
mk_comp('Input', 'input')

In [ ]:
@rt
def proc_inp(inp:str): return f'hi {inp}'

@rt
def testinp():
    return Form(hx_post=proc_inp, hx_target='#test')(
        Input(type='text', placeholder='Type here', id='inp'),
        Button('say hi', type='submit'),
        Div(id='test')
    )

In [ ]:
preview(testinp())

In [ ]:
form_jinja = jinja2.Template('''<form enctype="multipart/form-data" hx-post="/proc_inp" hx-target="#test">  <input type="text" placeholder="Type here" id="inp" class="input " name="inp">
<button type="submit" class="btn ">say hi</button>  <div id="test"></div>
</form>''')

In [ ]:
@rt
def testinp_jinja():
    return form_jinja.ft_render()

In [ ]:
preview(testinp_jinja())

In [ ]:
def nameinp(oob=False):
    kw = dict(hx_swap_oob='true') if oob else {}
    return Input(type='text', placeholder='Type here', id='inp', **kw)

@rt
def proc_inp(inp:str): 
    return P(f'hi {inp}'), nameinp(True)

@rt
def testinp():
    return Form(hx_post=proc_inp, hx_target='#test', hx_swap='afterbegin')(
        nameinp(),
        Button('say hi', type='submit'),
        Div(id='test')
    )

In [ ]:
preview(testinp())

In [ ]:
def nameinp_jinja(oob=False):
    kw = dict(hx_swap_oob='true') if oob else {}
    return jinja2.Template('''
    <input type="text" placeholder="Type here" id="inp" class="input" name="inp" {% for key, value in kw.items() %}{{key}}="{{value}}"{% endfor %}>
''').ft_render(kw=kw)

@rt
def proc_inp_jinja(inp:str): 
    return jinja2.Template("<p> hi {{inp}}").ft_render(inp = inp), nameinp_jinja(True)

@rt
def testinp_jinja():
    return jinja2.Template('''<form enctype="multipart/form-data" hx-post="/proc_inp_jinja" hx-target="#test", hx_swap="afterbegin">  <input type="text" placeholder="Type here" id="inp" class="input " name="inp">
<button type="submit" class="btn ">say hi</button>  <div id="test"></div>
</form>''').ft_render()

In [ ]:
preview(testinp_jinja())

### Jinja style

I will now try how I use Jinja with FastHTML, reading a file and checking inheritance.

#### From a file

In [ ]:
jinja_template = """<div> {{ name }} </div>
"""
jinja_template

'<div> {{ name }} </div>\n'

In [ ]:
%mkdir templates

In [ ]:
with open("templates/example.html", "w") as f:
    f.write(jinja_template)

In [ ]:
%ls templates/

example.html


I need to load the files from the environment

In [ ]:
env = jinja2.Environment(loader=jinja2.FileSystemLoader('templates'))

In [ ]:
template_from_file = env.get_template('example.html')
template_from_file

<Template 'example.html'>

In [ ]:
@rt
def example_from_file(name):
    return template_from_file.ft_render(name=name)

In [ ]:
preview(example_from_file(name="Yann"))

#### Inheritance

To correctly load the jinja templates, they should be in the environment, so jinja knows what template exits. 

In addition to load by folder, I am adding a DictLoader.

In [ ]:
parent = """<!DOCTYPE html>
<html lang="en">
<head>
    {% block head %}
    <link rel="stylesheet" href="style.css" />
    <title>{% block title %}{% endblock %} - My Webpage</title>
    {% endblock %}
</head>
<body>
    <div id="content">{% block content %}{% endblock %}</div>
    <div id="footer">
        {% block footer %}
        &copy; Copyright 2008 by <a href="http://domain.invalid/">you</a>.
        {% endblock %}
    </div>
</body>
</html>
"""

In [ ]:
child = """{% extends "parent.html" %}
{% block title %}Index{% endblock %}
{% block head %}
    {{ super() }}
    <style type="text/css">
        .important { color: #336699; }
    </style>
{% endblock %}
{% block content %}
    <h1>Index</h1>
    <p class="important">
      Welcome to my awesome homepage.
    </p>
{% endblock %}
"""

In [ ]:
templates_dict={}

env = jinja2.Environment(loader=jinja2.ChoiceLoader([
    jinja2.DictLoader(templates_dict),
    jinja2.FileSystemLoader('templates')
]))


In [ ]:
templates_dict["parent.html"]=parent
templates_dict

{'parent.html': '<!DOCTYPE html>\n<html lang="en">\n<head>\n    {% block head %}\n    <link rel="stylesheet" href="style.css" />\n    <title>{% block title %}{% endblock %} - My Webpage</title>\n    {% endblock %}\n</head>\n<body>\n    <div id="content">{% block content %}{% endblock %}</div>\n    <div id="footer">\n        {% block footer %}\n        &copy; Copyright 2008 by <a href="http://domain.invalid/">you</a>.\n        {% endblock %}\n    </div>\n</body>\n</html>\n'}

In [ ]:
parent_tp = env.get_template('parent.html')
parent_tp.ft_render()

```html
<html lang="en">

  <head>

    <link rel="stylesheet" href="style.css"></link>

    <title> - My Webpage</title>

  </head>

  <body>

    <div id="content"></div>

    <div id="footer">

        
        © Copyright 2008 by <a href="http://domain.invalid/">you</a>.
        
        </div>

  </body>

</html>

```

In [ ]:
preview(parent_tp.ft_render())

In [ ]:
templates_dict['child.html'] = child


In [ ]:
child_tp = env.get_template('child.html')
child_tp.ft_render()

```html
<html lang="en">

  <head>

    <link rel="stylesheet" href="style.css"></link>

    <title>Index - My Webpage</title>

    <style type="text/css">
        .important { color: #336699; }
    </style>

  </head>

  <body>

    <div id="content">

      <h1>Index</h1>

      <p class="important">
      Welcome to my awesome homepage.
    </p>

    </div>

    <div id="footer">

        
        © Copyright 2008 by <a href="http://domain.invalid/">you</a>.
        
        </div>

  </body>

</html>

```

In [ ]:
@rt
def child():
    return child_tp.ft_render()

In [ ]:
preview(child())

#### Correctly importing templates from code for inheritance

In [ ]:
#| export
templates_dict={}

env = jinja2.Environment(loader=jinja2.ChoiceLoader([
    jinja2.DictLoader(templates_dict),
    jinja2.FileSystemLoader('templates')
]))


In [ ]:
#| export

def add_template_from_code(
    templates_dict:dict[str], 
    code:str, 
    filename:str) :
    "Correct way to add a Jinja template to get inheritance"
    
    templates_dict[filename] = code
    return env.get_template(filename)

## Exporting templates like nbdev

All of this would not be worth if I could not export templates as I can export with nbdev. 

My current hack is to have a tag `#| exportjinja namefile.html` and use `find_msgs` from `dialoghelper`  to find those

In [ ]:
#| exportjinja parent.html
parent = """<!DOCTYPE html>
<html lang="en">
<head>
    {% block head %}
    <link rel="stylesheet" href="style.css" />
    <title>{% block title %}{% endblock %} - My Webpage</title>
    {% endblock %}
</head>
<body>
    <div id="content">{% block content %}{% endblock %}</div>
    <div id="footer">
        {% block footer %}
        &copy; Copyright 2008 by <a href="http://domain.invalid/">you</a>.
        {% endblock %}
    </div>
</body>
</html>
"""

In [ ]:
#| export
from dialoghelper import find_msgs

In [ ]:
x = find_msgs(re_pattern="#\\| exportjinja", msg_type="code")[0]["content"]
x

'#| exportjinja parent.html\nparent = """<!DOCTYPE html>\n<html lang="en">\n<head>\n    {% block head %}\n    <link rel="stylesheet" href="style.css" />\n    <title>{% block title %}{% endblock %} - My Webpage</title>\n    {% endblock %}\n</head>\n<body>\n    <div id="content">{% block content %}{% endblock %}</div>\n    <div id="footer">\n        {% block footer %}\n        &copy; Copyright 2008 by <a href="http://domain.invalid/">you</a>.\n        {% endblock %}\n    </div>\n</body>\n</html>\n"""'

In [ ]:
first_line = x.split('\n')[0]
file_name = first_line.split("exportjinja")[-1].strip()
file_name

'parent.html'

In [ ]:
content = x.split('"""')[1]
content

'<!DOCTYPE html>\n<html lang="en">\n<head>\n    {% block head %}\n    <link rel="stylesheet" href="style.css" />\n    <title>{% block title %}{% endblock %} - My Webpage</title>\n    {% endblock %}\n</head>\n<body>\n    <div id="content">{% block content %}{% endblock %}</div>\n    <div id="footer">\n        {% block footer %}\n        &copy; Copyright 2008 by <a href="http://domain.invalid/">you</a>.\n        {% endblock %}\n    </div>\n</body>\n</html>\n'

In [ ]:
with open(f"templates/{file_name}", "w") as f:
        f.write(content)

In [ ]:
%ls templates

child.html  example.html  parent.html


In [ ]:
#| export
def export_jinja_template(cell:str, templates_folder:str ="templates"):
    "Export single cell"
    first_line = cell.split('\n')[0]
    file_name = first_line.split("exportjinja")[-1].strip()
    
    if len(cell.split('"""')) == 3:
        content = cell.split('"""')[1]

    elif len(cell.split("'''")) == 3:
        content = cell.split("'''")[1]

    else:
        print("cannot find")
        return None
    
    with open(f"{templates_folder}/{file_name}", "w") as f:
        f.write(content)


In [ ]:
%rm templates/parent.html

In [ ]:
%ls templates

child.html  example.html


In [ ]:
export_jinja_template(x)

In [ ]:
%ls templates

child.html  example.html  parent.html


In [ ]:
!cat templates/parent.html

<!DOCTYPE html>
<html lang="en">
<head>
    {% block head %}
    <link rel="stylesheet" href="style.css" />
    <title>{% block title %}{% endblock %} - My Webpage</title>
    {% endblock %}
</head>
<body>
    <div id="content">{% block content %}{% endblock %}</div>
    <div id="footer">
        {% block footer %}
        &copy; Copyright 2008 by <a href="http://domain.invalid/">you</a>.
        {% endblock %}
    </div>
</body>
</html>


In [ ]:
x = find_msgs(re_pattern="#\\| exportjinja", msg_type="code")[0]["content"]
x

'#| exportjinja parent.html\nparent = """<!DOCTYPE html>\n<html lang="en">\n<head>\n    {% block head %}\n    <link rel="stylesheet" href="style.css" />\n    <title>{% block title %}{% endblock %} - My Webpage</title>\n    {% endblock %}\n</head>\n<body>\n    <div id="content">{% block content %}{% endblock %}</div>\n    <div id="footer">\n        {% block footer %}\n        &copy; Copyright 2008 by <a href="http://domain.invalid/">you</a>.\n        {% endblock %}\n    </div>\n</body>\n</html>\n"""'

In [ ]:
#| export
def export_jinja_templates():
    " Export all jinja template cells"
    cells_to_export = find_msgs(re_pattern="#\\| exportjinja", msg_type="code")
    [export_jinja_template(cell["content"]) for cell in cells_to_export]

In [ ]:
#| exportjinja child.html
child = """{% extends "parent.html" %}
{% block title %}Index{% endblock %}
{% block head %}
    {{ super() }}
    <style type="text/css">
        .important { color: #336699; }
    </style>
{% endblock %}
{% block content %}
    <h1>Index</h1>
    <p class="important">
      Welcome to my awesome homepage.
    </p>
{% endblock %}
"""

In [ ]:
%rm templates/parent.html

In [ ]:
%ls templates

child.html  example.html


In [ ]:
export_jinja_templates()

In [ ]:
%ls templates

child.html  example.html  parent.html


In [ ]:
!cat templates/parent.html

<!DOCTYPE html>
<html lang="en">
<head>
    {% block head %}
    <link rel="stylesheet" href="style.css" />
    <title>{% block title %}{% endblock %} - My Webpage</title>
    {% endblock %}
</head>
<body>
    <div id="content">{% block content %}{% endblock %}</div>
    <div id="footer">
        {% block footer %}
        &copy; Copyright 2008 by <a href="http://domain.invalid/">you</a>.
        {% endblock %}
    </div>
</body>
</html>


In [ ]:
!cat templates/child.html

{% extends "parent.html" %}
{% block title %}Index{% endblock %}
{% block head %}
    {{ super() }}
    <style type="text/css">
        .important { color: #336699; }
    </style>
{% endblock %}
{% block content %}
    <h1>Index</h1>
    <p class="important">
      Welcome to my awesome homepage.
    </p>
{% endblock %}


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)